### Generating models for the fitter

In [ ]:
import ROOT

In [ ]:
Bmass       = ROOT.RooRealVar("Bmass","m_{#phi#mu#mu} [GeV]", 4.5, 6.0)
CosThetaK   = ROOT.RooRealVar("CosThetaK", "cos#theta_{K}", -1., 1.)
CosThetaL   = ROOT.RooRealVar("CosThetaL", "cos#theta_{l}", -1., 1.)

In [ ]:
wspace = ROOT.RooWorkspace('wspace')

In [ ]:
# Legendre's Polynomials
xTerm = "(x0+x1*CosThetaK+x2*(1.5*pow(CosThetaK,2)-0.5)+x3*(2.5*pow(CosThetaK,3)-1.5*CosThetaK)+x4*(4.375*pow(CosThetaK, 4)-3.75*pow(CosThetaK, 2)+0.375))\
+(x5+x6*CosThetaK+x7*(1.5*pow(CosThetaK,2)-0.5)+x8*(2.5*pow(CosThetaK,3)-1.5*CosThetaK)+x9*(4.375*pow(CosThetaK, 4)-3.75*pow(CosThetaK, 2)+0.375))*CosThetaL\
+(x10+x11*CosThetaK+x12*(1.5*pow(CosThetaK,2)-0.5)+x13*(2.5*pow(CosThetaK,3)-1.5*CosThetaK)+x14*(4.375*pow(CosThetaK, 4)-3.75*pow(CosThetaK, 2)+0.375))*pow(CosThetaL,2)\
+(x15+x16*CosThetaK+x17*(1.5*pow(CosThetaK,2)-0.5)+x18*(2.5*pow(CosThetaK,3)-1.5*CosThetaK)+x19*(4.375*pow(CosThetaK, 4)-3.75*pow(CosThetaK, 2)+0.375))*pow(CosThetaL,3)\
+(x20+x21*CosThetaK+x22*(1.5*pow(CosThetaK,2)-0.5)+x23*(2.5*pow(CosThetaK,3)-1.5*CosThetaK)+x24*(4.375*pow(CosThetaK, 4)-3.75*pow(CosThetaK, 2)+0.375))*pow(CosThetaL,4)"

In [ ]:
# Building efficiency PDF

eff = [f"l{i}[-10,10]" for i in range(1, 8)]
eff += [f"k{i}[-10,10]" for i in range(1, 7)]
eff += ["RooPolynomial::effi_cosl(CosThetaL, {l1, l2, l3, l4, l5, l6, l7})"]
eff += ["RooPolynomial::effi_cosK(CosThetaK, {k1, k2, k3, k4, k5, k6})"]
eff += ["hasXTerm[0]"]
eff += ["expr::effi_xTerm('1+hasXTerm*({xTerm})',{args})"
        .format(
        xTerm=xTerm,
        args="{CosThetaL,CosThetaK,hasXTerm," + ','.join(["x{0}[-30,30]".format(i) for i in range(25)]) + "}"
        )
       ]
eff += ["prod::effi_sigA(effi_norm[0.5,0,1], effi_cosl, effi_cosK, effi_xTerm)"]

In [ ]:
wspace.Import(Bmass)
wspace.Import(CosThetaK)
wspace.Import(CosThetaL)
for obj in eff: wspace.factory(obj)

In [ ]:
# Lets import model built for B -. sll process
ROOT.gInterpreter.ProcessLine('#include "../cpp/RooBtosllModel.cpp"')

In [ ]:
# import cpp
# wspace.addClassImplImportDir('./')
for obj in ["unboundFl[0.6978,-3e3,3e3]", "unboundAfb[0.00,-3e4,3e4]"]: wspace.factory(obj)
f_sigA = ROOT.RooBtosllModel("f_sigA", "", CosThetaL, CosThetaK, wspace.var('unboundAfb'), wspace.var('unboundFl'))
wspace.Import(f_sigA)
# wspace.importClassCode(ROOT.RooBtosllModel.Class(), True)

In [ ]:
# Model for Bmass PDF

mass_pdf =  ["RooJohnson::f_sigM1(Bmass, sigM_mu[5.31, 5.25, 5.45], sigM_lambda1[0.96, 1e-6, 1], sigM_gamma1[-6.03, -10, 10], sigM_delta1[1.30, 1e-6, 10])"]
mass_pdf += ["RooJohnson::f_sigM2(Bmass, sigM_mu, sigM_lambda2[0.36, 1e-6, 1], sigM_gamma2[-0.75, -10, 10], sigM_delta2[9.27, 1e-6, 10])"]
mass_pdf += ["SUM::f_sigM(sigM_frac[0.00517, 0.,1.]*f_sigM1, f_sigM2)"]

for obj in mass_pdf: wspace.factory(obj)

In [ ]:
# Model 3D Component of Signal PDF

wspace.factory("RooEffProd::f_sig2D(f_sigA, effi_sigA)")
# effi_sigA = wspace.var('effi_sigA')
# f_sig2D  = ROOT.RooEffProd("f_sig2D", "", locals()['f_sigA'], locals()['effi_sigA'])
# wspace.Import(f_sig2D)
wspace.factory("PROD::f_sig3D(f_sigM, f_sig2D)")
# print(a)
# wspace.Print()

#### Saving workspace containing PDFs to the file

In [ ]:
#Saving workspace to the file
# out_file = ROOT.TFile('signal_models.root', 'recreate')
# out_file.WriteObject(wspace, 'wspace')
# out_file.Close()

In [ ]:
# wfile = ROOT.TFile('signal_models.root')
# newspace = wfile.Get('wspace')
# for i in newspace.allPdfs(): print(i)

In [ ]:
wspace.Print()

### Efficiency fit

In [ ]:
import os
file_path = '../files'
eff_file_name = os.path.join(file_path, 'accXrecEffHists_2016_bin3.root')
eff_file = ROOT.TFile(eff_file_name)

In [ ]:
dataX = eff_file.Get('h_accXrec_betweenPeaks_ProjectionX')
dataY = eff_file.Get('h_accXrec_betweenPeaks_ProjectionY')

In [ ]:
data_hist = eff_file.Get('h2_accXrec_betweenPeaks')
data = ROOT.RooDataHist("accXrec", "", ROOT.RooArgList(CosThetaL, CosThetaK), ROOT.RooFit.Import(data_hist))

In [ ]:
pdf = wspace.function('effi_sigA')
pdfX = wspace.function('effi_cosl')
pdfY = wspace.function('effi_cosK')

In [ ]:
args = pdf.getParameters(data)

In [ ]:
for arg in args: arg.setConstant(True)

In [ ]:
args.find('hasXTerm').setVal(0)

In [ ]:
import re
for proj, pdf_proj, var, pattern in [(dataX, pdfX, CosThetaL, r"^l\d+\w*"), (dataY, pdfY, CosThetaK, r"^k\d+\w*")]:
    hdata = ROOT.RooDataHist("hdata", "", ROOT.RooArgList(var), ROOT.RooFit.Import(proj))
    
    # Freeze all parameters parameters except those matching 'pattern'
    for arg in args: 
        arg.setConstant(False if re.match(pattern, arg.GetName()) else True)

    print(pdf)
    result = pdf_proj.fitTo(hdata, ROOT.RooFit.Save())
    result.Print("v")
                           

#### Fix normalization term

In [ ]:
for arg in args: arg.setConstant(True)
args.find('effi_norm').setConstant(False)
result = pdf.chi2FitTo(data, 
                       ROOT.RooFit.Minos(True), 
                       ROOT.RooFit.Save(), 
                       ROOT.RooFit.Strategy(2), 
                       ROOT.RooFit.PrintLevel(1))

### Do fitting for Cross-Term

In [ ]:
# Activate cross term
args.find('hasXTerm').setVal(1)
# Set cross-term parameters to floating
for arg in args: arg.setConstant(False if re.match(r"^x\d+\w*", arg.GetName()) else True)
# for i in args: print(i, i.isConstant()) if i.isConstant() else print('')

In [ ]:
new_result = pdf.chi2FitTo(data, 
                           ROOT.RooFit.Strategy(2), 
                           ROOT.RooFit.Save(1), 
                           ROOT.RooFit.PrintLevel(-1))

### Plotting efficiency distributions after fit

In [ ]:
import itertools
canvas = ROOT.TCanvas()
latex = ROOT.TLatex()
effi_2d = data_hist.Clone("effi_2d")
effi_2d.Reset("ICESM")
pdfhist = data_hist.Clone("pdfhist")
pdfhist.Reset("ICESM")
pdf.fillHistogram(pdfhist, ROOT.RooArgList(CosThetaL,CosThetaK))

for lBin, KBin in itertools.product(list(range(1, effi_2d.GetNbinsX() + 1)), list(range(1, effi_2d.GetNbinsY() + 1))):
    if data_hist.GetBinContent(lBin, KBin)==0:                                                                                
        effi_2d.SetBinContent(lBin, KBin, 0)                                                                           
        print (">> ** Warning ** Empty bins: (l, k)", lBin, KBin)                                                              
    else:            
        ratio = pdfhist.GetBinContent(lBin, KBin) / data_hist.GetBinContent(lBin, KBin)
        effi_2d.SetBinContent(lBin, KBin, ratio)
        
effi_2d.Draw("LEGO2")    
canvas.Draw()

#### Chi Square / NDF

In [ ]:
chi2ndf = new_result.minNll() / (
    data_hist.GetNbinsX()*data_hist.GetNbinsY() - \
    pdf.getParameters(data).selectByAttrib("Constant",0).getSize()
)
print("Chi-Square per NDF: ", chi2ndf)

In [ ]:
# cosK_hist = pdf.createHistogram("cosK_hist", CosThetaL, 
# ROOT.RooFit.Binning(40), 
# ROOT.RooFit.YVar(CosThetaK, ROOT.RooFit.Binning(40)))

# c2 = ROOT.TCanvas()
# dataL = ROOT.RooDataHist("RooHist_for_cosL", "", 
#                  ROOT.RooArgList(CosThetaL), 
#                  ROOT.RooFit.Import(dataX))
# print(type(dataL))
# frameL = CosThetaL.frame()
# dataL.plotOn(frameL, ROOT.RooFit.Rescale(100))
# c2.Draw()
# dataL.Print()